# Task 2: Recommendation Engine - Skeleton Notebook

This notebook provides a very basic example for the notebook you are expected to submit for Task 2 of the Final Project. The main purpose is that we can try different examples to get a better sense of your approach. Compared to Task 1 (Kaggle Competition), we don't have any objective means to evaluate the recommendations. 

Some general comments:
* You can import any data you need. This particularly includes your cleaned version of the properties dataset (incl. the auxiliary data or any other data you might have collected); there's no need to show the data cleaning / preprocessing steps in this notebook.
* You can also import your code in form of external Python (.py) script. You're actually encouraged to do so to keep this notebook light and uncluttered.
* **Important:** Please consider this notebook as an example and not to set specific requirements. Your notebook is likely to look very different. As long there is a section where we can easily test your solution, it should be fine.

## Setting up the Notebook

In [1]:
import pandas as pd
from data_processing import data_processing


%load_ext autoreload
%autoreload 2


## Load the Data

For this example, we use a simplified version of the dataset with only 2k+ data samples, each with only a subset of features.

In [2]:
import pandas as pd
import numpy as np
from data_processing import data_processing

df_sample = pd.read_csv('data/sg-property-prices-simplified.csv')

df_sample.head()

X = data_processing(df_sample)

X.head()



FileNotFoundError: [Errno 2] No such file or directory: 'data/sg-property-prices-simplified.csv'

## Computing the Top Recommendations

The method `get_top_recommendations()` shows an example of how to get the top recommendations for a given data sample (data sample = row in the dataframe of the dataset). The input is a row from the dataset and a list of optional input parameters which will depend on your approach; `k` is the number of returned recommendations seems useful, though.

The output should be a `pd.DataFrame` containing the recommendations. The output dataframe should have the same columns as the row + any additional columns you deem important (e.g., any score or tags that you might want to add to your recommendations).

In principle, the method `get_top_recommendations()` may be imported from a external Python (.py) script as well.

In [121]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import Series
from sklearn.metrics.pairwise import cosine_similarity

def cal_sim(x, y):
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

def calculate_sim(car_array, item):
    sim = np.array([cal_sim(car_array[i], item) for i in range(0,len(car_array))])
    return sim

def get_user_profile(listingid_score_dict, processed_df, non_zero_column_num):
    scores = np.array(list(listingid_score_dict.values()))
    scores_norm = (scores - scores.mean()).T
    rated_items = processed_df.set_index('listing_id').loc[list(listingid_score_dict.keys())].reset_index(inplace=False).drop(columns=['listing_id'])
    user_profile = scores_norm.dot(rated_items) / non_zero_column_num
    return user_profile

# Task2-track2: compute cosine similarity for all items
def calculate_cos_similar_for_all_items(user_profile, user_item_index, df):
    similarity_list = []
    user_profile = np.array(user_profile).reshape(1,user_profile.shape[0])
    unrated_items = df[~df['listing_id'].isin(user_item_index)]
    for index, row in unrated_items.iterrows():
        row_cleaned = np.array(row[1:]).reshape(1,user_profile.shape[1])
        simi = cosine_similarity(row_cleaned, user_profile)
        t = (simi[0][0], row['listing_id'])
        similarity_list.append(t)
    return similarity_list


def get_top_recommendations(k, **kwargs):
    filtered_data = kwargs.get("filtered_data")
    listingid_score_dict = kwargs.get("listingid_score_dict")
    
    # filtered data processing: normalize and one-hot encode
    processed_df = X
    
    # get the number of features

    non_zero_column_num = filtered_data.shape[1]-1
    
    # get user profile
    user_profile = get_user_profile(listingid_score_dict, processed_df, non_zero_column_num)
    
    # compute similarity for all items in filtered data
    similarity_list = calculate_cos_similar_for_all_items(user_profile=user_profile, user_item_index=list(listingid_score_dict.keys()), df=processed_df)
    sorted_list = sorted(similarity_list,key=lambda t:t[0], reverse=True)
    
    # return top k items
    top_k = sorted_list[:k]
    id_list = []
    for x in top_k:
        id_list.append(x[1])
    recommend_items = X.set_index('listing_id').loc[id_list].reset_index(inplace=False)
    return recommend_items





## Testing the Recommendation Engine

This will be the main part of your notebook to allow for testing your solutions. Most basically, for a given listing (defined by the row id in your input dataframe), we would like to see the recommendations you make. So however you set up your notebook, it should have at least a comparable section that will allow us to run your solution for different inputs.

### Pick a Sample Listing as Input

In [122]:
user_select_data = X.head()



# generate scores--discussed in the morning
n = user_select_data.shape[0]
user_item = user_select_data.sample(n=n)

user_item_index = user_item['listing_id'].tolist()

scores = [random.randint(-2,2) for _ in range(n)]


listingid_score_dict = {user_item_index[i]: scores[i] for i in range(5)}


print(f"Your score list is: {listingid_score_dict}")
X[X['listing_id'].isin(listingid_score_dict.keys())]




Your score list is: {252293: 2, 799762: -1, 926453: 1, 445021: 0, 896907: 1}


,num_beds,num_baths,size_sqft,area_mean_price,listing_id,property_type_condo,property_type_hdb,property_type_house,built_year_1995,built_year_2005,built_year_2015,built_year_2025,price_2.0,price_3.0,price_4.0
0,-0.830249,-0.385384,-0.469722,-0.343703,799762,0,1,0,0,0,0,0,0,0,0
1,-0.830249,-0.385384,-0.476024,-0.343703,896907,0,1,0,0,0,0,0,0,0,0
2,-0.013897,-0.385384,-0.347105,-0.343703,445021,0,1,0,0,0,0,0,0,0,0
3,-0.013897,-0.385384,-0.285224,-0.343703,252293,0,1,0,0,0,0,0,0,0,0
4,-0.013897,-0.385384,-0.315591,-0.343703,926453,0,1,0,0,0,1,0,0,0,0


## Compute and Display the recommendations

Since the method `get_top_recommendations()` returns a `pd.DataFrame`, it's easy to display the result.

In [123]:
get_top_recommendations(3, filtered_data=user_select_data, listingid_score_dict=listingid_score_dict)

,listing_id,num_beds,num_baths,size_sqft,area_mean_price,property_type_condo,property_type_hdb,property_type_house,built_year_1995,built_year_2005,built_year_2015,built_year_2025,price_2.0,price_3.0,price_4.0
0,118782,1.618805,2.554029,4.267634,0.922607,0,0,0,0,0,1,0,0,0,1
1,123048,3.251508,3.288883,3.621318,-0.354482,0,0,1,1,0,0,0,0,0,1
2,207219,4.884211,4.758590,4.840608,1.508647,0,0,0,1,0,0,0,0,0,1
